# Workshop: Genomics of Identity-By-Descent (IBD) Segments

Welcome to this practical workshop on IBD segment analysis! This notebook will guide you through three main topics:

1. **Identification of IBD segments**
2. **Temporal demographic inference**
3. **Spatial demographic inference**

We'll use population genetic simulations to generate data and explore these concepts interactively.

## Setup: Installing Required Tools

First, we need to install the required Python packages for running simulations. The main tools we'll use are:

- **msprime**: A coalescent simulator for generating genetic variation data
- **tskit**: A library for working with tree sequences
- **SLiM**: A forward-time population genetics simulator (we'll use pyslim to interface with it)

Run the following cell to install these packages:

In [ ]:
# Install required packages
# You can either install from requirements.txt:
!pip install -r requirements.txt
# Or install packages individually (may use different versions):
# !pip install msprime tskit pyslim numpy matplotlib pandas seaborn scipy ipywidgets

Now let's import the libraries we'll need:

In [ ]:
import msprime
import tskit
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from IPython.display import display, HTML
from ipywidgets import widgets, interact
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("All packages imported successfully!")
print(f"msprime version: {msprime.__version__}")
print(f"tskit version: {tskit.__version__}")

---
# Part 1: Identification of IBD Segments

## Introduction

**Identity By Descent (IBD)** refers to genomic segments that are identical between individuals because they were inherited from a common ancestor without recombination. IBD segments are powerful tools for:

- Inferring recent population history
- Detecting relatives
- Understanding population structure
- Fine-scale demographic inference

### Key Concepts:

1. **IBD segments** arise when two chromosomes share a continuous region inherited from a common ancestor
2. **Length of IBD segments** depends on the time to the most recent common ancestor (TMRCA)
3. **Longer segments** indicate more recent shared ancestry
4. **Shorter segments** indicate more distant ancestry (more time for recombination to break them up)

## Exercise 1.1: Simulating a Simple Population

Let's start by simulating a simple population and exploring its tree sequences. We'll use `msprime` to simulate genetic data under a coalescent model.

In [ ]:
# Simulate a simple population
def simulate_simple_population(sample_size=10, Ne=10000, length=1e6, recombination_rate=1e-8, random_seed=42):
    """
    Simulate a simple population using msprime.
    
    Parameters:
    - sample_size: number of diploid individuals to sample
    - Ne: effective population size
    - length: sequence length in base pairs
    - recombination_rate: per-base recombination rate
    - random_seed: random seed for reproducibility
    """
    ts = msprime.sim_ancestry(
        samples=sample_size,
        population_size=Ne,
        sequence_length=length,
        recombination_rate=recombination_rate,
        random_seed=random_seed
    )
    return ts

# Run the simulation
ts = simulate_simple_population(sample_size=20, Ne=10000, length=10e6)

print(f"Simulated tree sequence:")
print(f"  - Number of trees: {ts.num_trees}")
print(f"  - Number of samples: {ts.num_samples}")
print(f"  - Sequence length: {ts.sequence_length/1e6:.1f} Mb")
print(f"  - Number of nodes: {ts.num_nodes}")

### Visualizing Tree Sequences

Let's visualize a portion of the tree sequence to understand its structure:

In [ ]:
# Display a subset of trees
subset_ts = ts.keep_intervals([[0, 100000]], simplify=True)
display(HTML(subset_ts.draw_svg(y_axis=True, size=(800, 400))))

## Exercise 1.2: Computing IBD Segments

Now let's identify IBD segments between pairs of individuals. We'll use the tree sequence to find regions where samples share recent common ancestry.

In [ ]:
def find_ibd_segments(ts, max_time=500, min_length=100000):
    """
    Find IBD segments between pairs of samples.
    
    Parameters:
    - ts: tree sequence
    - max_time: maximum time depth (in generations) to consider for IBD
    - min_length: minimum length (in bp) for IBD segments
    
    Returns:
    - DataFrame with IBD segments
    """
    ibd_segments = []
    
    # Iterate through trees
    for tree in ts.trees():
        # Check all pairs of samples
        for i in range(ts.num_samples):
            for j in range(i+1, ts.num_samples):
                # Find MRCA of the pair
                mrca = tree.mrca(i, j)
                if mrca != tskit.NULL:
                    mrca_time = tree.time(mrca)
                    if mrca_time <= max_time:
                        segment_length = tree.interval.right - tree.interval.left
                        if segment_length >= min_length:
                            ibd_segments.append({
                                'sample1': i,
                                'sample2': j,
                                'start': tree.interval.left,
                                'end': tree.interval.right,
                                'length': segment_length,
                                'tmrca': mrca_time
                            })
    
    return pd.DataFrame(ibd_segments)

# Find IBD segments
ibd_df = find_ibd_segments(ts, max_time=1000, min_length=50000)

print(f"Found {len(ibd_df)} IBD segments")
print("\nFirst few IBD segments:")
display(ibd_df.head(10))

### Visualizing IBD Segment Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: IBD length distribution
axes[0].hist(ibd_df['length']/1e6, bins=30, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('IBD Segment Length (Mb)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of IBD Segment Lengths')

# Plot 2: TMRCA vs length
axes[1].scatter(ibd_df['tmrca'], ibd_df['length']/1e6, alpha=0.5)
axes[1].set_xlabel('TMRCA (generations)')
axes[1].set_ylabel('IBD Segment Length (Mb)')
axes[1].set_title('IBD Length vs. Time to MRCA')

plt.tight_layout()
plt.show()

## Interactive Question 1.1

**What is the relationship between IBD segment length and TMRCA?**

In [ ]:
# Interactive multiple choice question
question_1_1 = widgets.RadioButtons(
    options=[
        'Longer segments indicate more ancient ancestry',
        'Longer segments indicate more recent ancestry',
        'There is no relationship between length and TMRCA',
        'Segment length depends only on population size'
    ],
    description='Answer:',
    disabled=False
)

def check_answer_1_1(b):
    if question_1_1.value == 'Longer segments indicate more recent ancestry':
        print("✓ Correct! Longer IBD segments indicate more recent common ancestry because there has been ")
        print("  less time for recombination to break them up.")
    else:
        print("✗ Not quite. Think about what happens to IBD segments over time due to recombination.")

button_1_1 = widgets.Button(description="Submit Answer")
button_1_1.on_click(check_answer_1_1)

display(question_1_1, button_1_1)

## Exercise 1.3: Effect of Recombination Rate

Let's explore how recombination rate affects the number and length of IBD segments.

In [ ]:
# Simulate with different recombination rates
recomb_rates = [1e-9, 1e-8, 1e-7]
results = []

for rate in recomb_rates:
    ts_temp = simulate_simple_population(sample_size=20, Ne=10000, length=10e6, 
                                         recombination_rate=rate, random_seed=42)
    ibd_temp = find_ibd_segments(ts_temp, max_time=1000, min_length=50000)
    results.append({
        'rate': rate,
        'num_segments': len(ibd_temp),
        'mean_length': ibd_temp['length'].mean() if len(ibd_temp) > 0 else 0,
        'num_trees': ts_temp.num_trees
    })

results_df = pd.DataFrame(results)
print("Effect of recombination rate:")
display(results_df)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(results_df['rate'], results_df['num_segments'], marker='o', linewidth=2)
axes[0].set_xlabel('Recombination Rate (per bp per generation)')
axes[0].set_ylabel('Number of IBD Segments')
axes[0].set_xscale('log')
axes[0].set_title('IBD Segments vs. Recombination Rate')

axes[1].plot(results_df['rate'], results_df['mean_length']/1e6, marker='o', linewidth=2, color='orange')
axes[1].set_xlabel('Recombination Rate (per bp per generation)')
axes[1].set_ylabel('Mean IBD Length (Mb)')
axes[1].set_xscale('log')
axes[1].set_title('Mean IBD Length vs. Recombination Rate')

plt.tight_layout()
plt.show()

## Interactive Question 1.2

**How does recombination rate affect IBD segment detection?**

In [ ]:
question_1_2 = widgets.RadioButtons(
    options=[
        'Higher recombination leads to fewer, longer IBD segments',
        'Higher recombination leads to more, shorter IBD segments',
        'Recombination has no effect on IBD segments',
        'Higher recombination eliminates all IBD segments'
    ],
    description='Answer:',
    disabled=False
)

def check_answer_1_2(b):
    if question_1_2.value == 'Higher recombination leads to more, shorter IBD segments':
        print("✓ Correct! Higher recombination breaks up ancestral segments more frequently, ")
        print("  resulting in more numerous but shorter IBD segments.")
    else:
        print("✗ Not quite. Think about how recombination breaks up chromosomal segments.")

button_1_2 = widgets.Button(description="Submit Answer")
button_1_2.on_click(check_answer_1_2)

display(question_1_2, button_1_2)

---
# Part 2: Temporal Demographic Inference

## Introduction

IBD segments can be used to infer **demographic history over time**. The distribution of IBD segment lengths contains information about:

- **Population size changes** (expansions, bottlenecks)
- **Migration events**
- **Recent demographic history** (last ~100-200 generations)

### Key Principle:

The rate of IBD sharing at different time depths reflects the effective population size at those times:
- **More IBD sharing** → smaller effective population size
- **Less IBD sharing** → larger effective population size

## Exercise 2.1: Simulating Population Size Changes

Let's simulate populations with different demographic histories and see how they affect IBD patterns.

In [ ]:
def simulate_bottleneck(sample_size=50, bottleneck_time=200, bottleneck_Ne=1000, 
                       ancient_Ne=10000, length=10e6, random_seed=42):
    """
    Simulate a population that experienced a bottleneck.
    """
    # Define demographic events
    demography = msprime.Demography()
    demography.add_population(name="A", initial_size=ancient_Ne)
    demography.add_population_parameters_change(
        time=bottleneck_time, population="A", initial_size=bottleneck_Ne
    )
    
    ts = msprime.sim_ancestry(
        samples={"A": sample_size},
        demography=demography,
        sequence_length=length,
        recombination_rate=1e-8,
        random_seed=random_seed
    )
    return ts

def simulate_expansion(sample_size=50, expansion_time=200, ancient_Ne=1000,
                      modern_Ne=10000, length=10e6, random_seed=42):
    """
    Simulate a population that experienced expansion.
    """
    demography = msprime.Demography()
    demography.add_population(name="A", initial_size=modern_Ne)
    demography.add_population_parameters_change(
        time=expansion_time, population="A", initial_size=ancient_Ne
    )
    
    ts = msprime.sim_ancestry(
        samples={"A": sample_size},
        demography=demography,
        sequence_length=length,
        recombination_rate=1e-8,
        random_seed=random_seed
    )
    return ts

# Simulate different scenarios
ts_constant = simulate_simple_population(sample_size=50, Ne=10000, length=10e6, random_seed=42)
ts_bottleneck = simulate_bottleneck(sample_size=50, random_seed=42)
ts_expansion = simulate_expansion(sample_size=50, random_seed=42)

print("Simulations completed!")
print(f"Constant size: {ts_constant.num_trees} trees")
print(f"Bottleneck: {ts_bottleneck.num_trees} trees")
print(f"Expansion: {ts_expansion.num_trees} trees")

## Exercise 2.2: Comparing IBD Patterns Across Demographic Scenarios

In [ ]:
# Find IBD segments for each scenario
ibd_constant = find_ibd_segments(ts_constant, max_time=2000, min_length=50000)
ibd_bottleneck = find_ibd_segments(ts_bottleneck, max_time=2000, min_length=50000)
ibd_expansion = find_ibd_segments(ts_expansion, max_time=2000, min_length=50000)

# Add scenario labels
ibd_constant['scenario'] = 'Constant'
ibd_bottleneck['scenario'] = 'Bottleneck'
ibd_expansion['scenario'] = 'Expansion'

# Combine data
ibd_all = pd.concat([ibd_constant, ibd_bottleneck, ibd_expansion])

print(f"IBD segments found:")
print(f"  Constant: {len(ibd_constant)}")
print(f"  Bottleneck: {len(ibd_bottleneck)}")
print(f"  Expansion: {len(ibd_expansion)}")

In [ ]:
# Visualize IBD distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Length distributions
for scenario in ['Constant', 'Bottleneck', 'Expansion']:
    data = ibd_all[ibd_all['scenario'] == scenario]['length']/1e6
    axes[0, 0].hist(data, bins=30, alpha=0.5, label=scenario)
axes[0, 0].set_xlabel('IBD Length (Mb)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('IBD Length Distribution')
axes[0, 0].legend()

# Plot 2: TMRCA distributions
for scenario in ['Constant', 'Bottleneck', 'Expansion']:
    data = ibd_all[ibd_all['scenario'] == scenario]['tmrca']
    axes[0, 1].hist(data, bins=30, alpha=0.5, label=scenario)
axes[0, 1].set_xlabel('TMRCA (generations)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('TMRCA Distribution')
axes[0, 1].legend()

# Plot 3: Length vs TMRCA (Constant)
axes[1, 0].scatter(ibd_constant['tmrca'], ibd_constant['length']/1e6, alpha=0.3)
axes[1, 0].set_xlabel('TMRCA (generations)')
axes[1, 0].set_ylabel('IBD Length (Mb)')
axes[1, 0].set_title('Constant Population')

# Plot 4: Length vs TMRCA (Bottleneck)
axes[1, 1].scatter(ibd_bottleneck['tmrca'], ibd_bottleneck['length']/1e6, alpha=0.3, color='orange')
axes[1, 1].set_xlabel('TMRCA (generations)')
axes[1, 1].set_ylabel('IBD Length (Mb)')
axes[1, 1].set_title('Bottleneck Population')

plt.tight_layout()
plt.show()

## Exercise 2.3: Inferring Population Size from IBD

We can estimate effective population size over time using the distribution of IBD segment lengths.

In [ ]:
def estimate_Ne_from_ibd(ibd_df, time_bins, sequence_length):
    """
    Estimate effective population size from IBD segments.
    
    Parameters:
    - ibd_df: DataFrame with IBD segments
    - time_bins: time bins for estimation
    - sequence_length: total sequence length
    """
    ne_estimates = []
    
    for i in range(len(time_bins)-1):
        t_start = time_bins[i]
        t_end = time_bins[i+1]
        
        # Count IBD segments in this time range
        mask = (ibd_df['tmrca'] >= t_start) & (ibd_df['tmrca'] < t_end)
        n_segments = mask.sum()
        
        if n_segments > 0:
            # Total IBD length in this bin
            total_length = ibd_df[mask]['length'].sum()
            
            # Estimate Ne (simplified, inversely proportional to IBD sharing)
            ne_est = sequence_length / (total_length + 1) * 5000  # scaling factor
            
            ne_estimates.append({
                'time_midpoint': (t_start + t_end) / 2,
                'Ne': ne_est,
                'n_segments': n_segments
            })
    
    return pd.DataFrame(ne_estimates)

# Define time bins
time_bins = np.linspace(0, 2000, 21)

# Estimate Ne for each scenario
ne_constant = estimate_Ne_from_ibd(ibd_constant, time_bins, ts_constant.sequence_length)
ne_bottleneck = estimate_Ne_from_ibd(ibd_bottleneck, time_bins, ts_bottleneck.sequence_length)

# Plot Ne estimates
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(ne_constant['time_midpoint'], ne_constant['Ne'], 'o-', label='Constant', linewidth=2)
ax.plot(ne_bottleneck['time_midpoint'], ne_bottleneck['Ne'], 's-', label='Bottleneck', linewidth=2)

# Add true values
ax.axhline(y=10000, color='blue', linestyle='--', alpha=0.5, label='True Ne (constant)')
ax.axhline(y=1000, color='orange', linestyle='--', alpha=0.5, label='True Ne (bottleneck)')

ax.set_xlabel('Time (generations ago)')
ax.set_ylabel('Estimated Ne')
ax.set_title('Effective Population Size Inference from IBD')
ax.set_yscale('log')
ax.legend()
ax.grid(True, alpha=0.3)

plt.show()

## Interactive Question 2.1

**In a population that experienced a recent bottleneck, what would you expect to see in the IBD data?**

In [ ]:
question_2_1 = widgets.RadioButtons(
    options=[
        'Fewer IBD segments overall',
        'More long IBD segments',
        'Only very short IBD segments',
        'No change in IBD patterns'
    ],
    description='Answer:',
    disabled=False
)

def check_answer_2_1(b):
    if question_2_1.value == 'More long IBD segments':
        print("✓ Correct! A bottleneck reduces diversity and causes individuals to share more ")
        print("  recent common ancestors, leading to more (and longer) IBD segments.")
    else:
        print("✗ Not quite. Think about what happens to genetic diversity during a bottleneck.")

button_2_1 = widgets.Button(description="Submit Answer")
button_2_1.on_click(check_answer_2_1)

display(question_2_1, button_2_1)

## Interactive Question 2.2

**What time period does IBD analysis primarily inform us about?**

In [ ]:
question_2_2 = widgets.RadioButtons(
    options=[
        'Ancient history (>10,000 generations ago)',
        'Recent history (~50-500 generations ago)',
        'Only the last 5 generations',
        'All time periods equally'
    ],
    description='Answer:',
    disabled=False
)

def check_answer_2_2(b):
    if question_2_2.value == 'Recent history (~50-500 generations ago)':
        print("✓ Correct! IBD segments are most informative about recent demographic history ")
        print("  because very old segments have been broken up by recombination, while very ")
        print("  recent segments are less common.")
    else:
        print("✗ Not quite. Consider which time periods produce detectable IBD segments.")

button_2_2 = widgets.Button(description="Submit Answer")
button_2_2.on_click(check_answer_2_2)

display(question_2_2, button_2_2)

---
# Part 3: Spatial Demographic Inference

## Introduction

IBD segments can also reveal **spatial population structure** and history:

- **Migration patterns** between populations
- **Isolation by distance**
- **Population splits and divergence**
- **Admixture events**

### Key Concepts:

1. **Within-population IBD**: Reflects local effective population size
2. **Between-population IBD**: Reflects migration and shared ancestry
3. **IBD sharing decreases with geographic distance** (isolation by distance)

## Exercise 3.1: Simulating Structured Populations

Let's simulate a population with geographic structure and migration.

In [ ]:
def simulate_two_populations(sample_size_per_pop=25, Ne=10000, split_time=1000,
                            migration_rate=1e-4, length=10e6, random_seed=42):
    """
    Simulate two populations that split in the past with ongoing migration.
    """
    demography = msprime.Demography()
    demography.add_population(name="Pop1", initial_size=Ne)
    demography.add_population(name="Pop2", initial_size=Ne)
    demography.add_population(name="Ancestral", initial_size=Ne)
    
    # Set migration rates
    demography.set_migration_rate(source="Pop1", dest="Pop2", rate=migration_rate)
    demography.set_migration_rate(source="Pop2", dest="Pop1", rate=migration_rate)
    
    # Population split
    demography.add_population_split(time=split_time, derived=["Pop1", "Pop2"], ancestral="Ancestral")
    
    ts = msprime.sim_ancestry(
        samples={"Pop1": sample_size_per_pop, "Pop2": sample_size_per_pop},
        demography=demography,
        sequence_length=length,
        recombination_rate=1e-8,
        random_seed=random_seed
    )
    return ts

# Simulate structured population
ts_structured = simulate_two_populations(sample_size_per_pop=25, migration_rate=1e-4, random_seed=42)

print("Structured population simulation completed!")
print(f"Number of samples: {ts_structured.num_samples}")
print(f"Number of trees: {ts_structured.num_trees}")
print(f"Number of populations: {ts_structured.num_populations}")

## Exercise 3.2: Analyzing Within- vs. Between-Population IBD

In [ ]:
def find_ibd_by_population(ts, max_time=1000, min_length=50000):
    """
    Find IBD segments and classify them by population pair type.
    """
    ibd_segments = []
    
    # Get population assignments
    # First half are Pop1, second half are Pop2
    n_samples = ts.num_samples
    pop1_samples = list(range(n_samples // 2))
    pop2_samples = list(range(n_samples // 2, n_samples))
    
    for tree in ts.trees():
        for i in range(ts.num_samples):
            for j in range(i+1, ts.num_samples):
                mrca = tree.mrca(i, j)
                if mrca != tskit.NULL:
                    mrca_time = tree.time(mrca)
                    if mrca_time <= max_time:
                        segment_length = tree.interval.right - tree.interval.left
                        if segment_length >= min_length:
                            # Determine population pair type
                            if i in pop1_samples and j in pop1_samples:
                                pair_type = 'Within Pop1'
                            elif i in pop2_samples and j in pop2_samples:
                                pair_type = 'Within Pop2'
                            else:
                                pair_type = 'Between Populations'
                            
                            ibd_segments.append({
                                'sample1': i,
                                'sample2': j,
                                'start': tree.interval.left,
                                'end': tree.interval.right,
                                'length': segment_length,
                                'tmrca': mrca_time,
                                'pair_type': pair_type
                            })
    
    return pd.DataFrame(ibd_segments)

# Find IBD segments by population
ibd_spatial = find_ibd_by_population(ts_structured, max_time=1500, min_length=50000)

print("IBD segments by population pair type:")
print(ibd_spatial['pair_type'].value_counts())
print("\nSummary statistics:")
display(ibd_spatial.groupby('pair_type')[['length', 'tmrca']].describe())

In [ ]:
# Visualize within vs between population IBD
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot 1: Length distribution by pair type
for pair_type in ['Within Pop1', 'Within Pop2', 'Between Populations']:
    data = ibd_spatial[ibd_spatial['pair_type'] == pair_type]['length']/1e6
    axes[0].hist(data, bins=30, alpha=0.6, label=pair_type)
axes[0].set_xlabel('IBD Length (Mb)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('IBD Length Distribution by Population Pair')
axes[0].legend()

# Plot 2: TMRCA distribution by pair type
for pair_type in ['Within Pop1', 'Within Pop2', 'Between Populations']:
    data = ibd_spatial[ibd_spatial['pair_type'] == pair_type]['tmrca']
    axes[1].hist(data, bins=30, alpha=0.6, label=pair_type)
axes[1].set_xlabel('TMRCA (generations)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('TMRCA Distribution by Population Pair')
axes[1].legend()

# Plot 3: Box plot of IBD lengths
ibd_spatial_plot = ibd_spatial.copy()
ibd_spatial_plot['length_mb'] = ibd_spatial_plot['length'] / 1e6
sns.boxplot(data=ibd_spatial_plot, x='pair_type', y='length_mb', ax=axes[2])
axes[2].set_xlabel('Population Pair Type')
axes[2].set_ylabel('IBD Length (Mb)')
axes[2].set_title('IBD Length Comparison')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Exercise 3.3: Effect of Migration Rate

Let's explore how migration rate affects IBD sharing between populations.

In [ ]:
# Simulate with different migration rates
migration_rates = [0, 1e-5, 1e-4, 1e-3]
migration_results = []

for mig_rate in migration_rates:
    ts_temp = simulate_two_populations(sample_size_per_pop=25, migration_rate=mig_rate, 
                                       random_seed=42)
    ibd_temp = find_ibd_by_population(ts_temp, max_time=1500, min_length=50000)
    
    # Count between-population IBD
    between_pop = ibd_temp[ibd_temp['pair_type'] == 'Between Populations']
    within_pop = ibd_temp[ibd_temp['pair_type'] != 'Between Populations']
    
    migration_results.append({
        'migration_rate': mig_rate,
        'between_pop_segments': len(between_pop),
        'within_pop_segments': len(within_pop),
        'ratio': len(between_pop) / len(within_pop) if len(within_pop) > 0 else 0,
        'mean_between_length': between_pop['length'].mean() if len(between_pop) > 0 else 0
    })

migration_results_df = pd.DataFrame(migration_results)
print("Effect of migration rate on IBD sharing:")
display(migration_results_df)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(migration_results_df['migration_rate'], 
             migration_results_df['between_pop_segments'], 
             marker='o', linewidth=2, markersize=8)
axes[0].set_xlabel('Migration Rate (per generation)')
axes[0].set_ylabel('Number of Between-Population IBD Segments')
axes[0].set_xscale('log')
axes[0].set_title('IBD Sharing vs. Migration Rate')
axes[0].grid(True, alpha=0.3)

axes[1].plot(migration_results_df['migration_rate'], 
             migration_results_df['ratio'], 
             marker='s', linewidth=2, markersize=8, color='orange')
axes[1].set_xlabel('Migration Rate (per generation)')
axes[1].set_ylabel('Between / Within IBD Ratio')
axes[1].set_xscale('log')
axes[1].set_title('Relative IBD Sharing Between Populations')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Exercise 3.4: Creating an IBD Sharing Matrix

A useful visualization for spatial structure is an IBD sharing matrix showing pairwise IBD between all individuals.

In [ ]:
def create_ibd_matrix(ts, ibd_df):
    """
    Create a matrix of total IBD sharing between all pairs of samples.
    """
    n_samples = ts.num_samples
    ibd_matrix = np.zeros((n_samples, n_samples))
    
    for _, row in ibd_df.iterrows():
        i, j = int(row['sample1']), int(row['sample2'])
        ibd_matrix[i, j] += row['length']
        ibd_matrix[j, i] += row['length']
    
    return ibd_matrix

# Create IBD matrix
ibd_matrix = create_ibd_matrix(ts_structured, ibd_spatial)

# Visualize
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(ibd_matrix / 1e6, cmap='YlOrRd', aspect='auto')

# Add population boundaries
n_samples = ts_structured.num_samples
mid_point = n_samples // 2
ax.axhline(y=mid_point-0.5, color='blue', linewidth=2, linestyle='--')
ax.axvline(x=mid_point-0.5, color='blue', linewidth=2, linestyle='--')

ax.set_xlabel('Sample ID')
ax.set_ylabel('Sample ID')
ax.set_title('Total IBD Sharing Matrix\n(Pop1: 0-24, Pop2: 25-49)')

# Add colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Total IBD (Mb)', rotation=270, labelpad=20)

plt.tight_layout()
plt.show()

## Interactive Question 3.1

**What does higher IBD sharing between populations indicate?**

In [ ]:
question_3_1 = widgets.RadioButtons(
    options=[
        'Complete isolation between populations',
        'Recent gene flow or shared ancestry',
        'Ancient divergence',
        'Different effective population sizes'
    ],
    description='Answer:',
    disabled=False
)

def check_answer_3_1(b):
    if question_3_1.value == 'Recent gene flow or shared ancestry':
        print("✓ Correct! Higher between-population IBD sharing indicates either recent ")
        print("  migration/gene flow or a recent split from a common ancestral population.")
    else:
        print("✗ Not quite. Think about what creates IBD segments between populations.")

button_3_1 = widgets.Button(description="Submit Answer")
button_3_1.on_click(check_answer_3_1)

display(question_3_1, button_3_1)

## Interactive Question 3.2

**In the IBD sharing matrix, what pattern indicates population structure?**

In [ ]:
question_3_2 = widgets.RadioButtons(
    options=[
        'Uniform IBD sharing across all pairs',
        'Blocks of higher sharing along the diagonal',
        'Random patterns with no structure',
        'Higher sharing in off-diagonal blocks'
    ],
    description='Answer:',
    disabled=False
)

def check_answer_3_2(b):
    if question_3_2.value == 'Blocks of higher sharing along the diagonal':
        print("✓ Correct! Blocks of higher IBD sharing along the diagonal indicate population ")
        print("  structure, with more sharing within populations than between populations.")
    else:
        print("✗ Not quite. Consider what an IBD matrix would look like with distinct populations.")

button_3_2 = widgets.Button(description="Submit Answer")
button_3_2.on_click(check_answer_3_2)

display(question_3_2, button_3_2)

---
# Summary and Conclusions

In this workshop, we've explored three main aspects of IBD segment analysis:

## 1. Identification of IBD Segments
- IBD segments are regions of the genome inherited from a common ancestor
- Segment length is inversely related to time to MRCA
- Recombination rate affects the number and length of detectable segments

## 2. Temporal Demographic Inference
- IBD segment distributions reflect population size changes over time
- Bottlenecks increase IBD sharing
- IBD is most informative for recent demographic history (~50-500 generations)

## 3. Spatial Demographic Inference
- Within-population IBD > between-population IBD indicates structure
- Migration increases between-population IBD sharing
- IBD matrices visualize population structure effectively

## Key Takeaways:
1. IBD segments are powerful tools for demographic inference
2. Both temporal and spatial aspects of demography leave signatures in IBD patterns
3. Simulations help us understand and interpret empirical IBD data
4. Multiple factors (Ne, migration, recombination) interact to shape IBD distributions

---
## Additional Exercises (Optional)

Try modifying the simulations to explore:

1. **Three-population models**: Add a third population and explore more complex migration patterns
2. **Continuous migration**: Simulate stepping-stone migration models
3. **Admixture events**: Add discrete admixture events and see their effect on IBD
4. **Variable recombination rates**: Use recombination maps with hotspots
5. **Selection**: Add positive or negative selection and observe effects on linked IBD segments

## References and Further Reading

- **msprime documentation**: https://tskit.dev/msprime/
- **tskit documentation**: https://tskit.dev/tskit/
- **SLiM documentation**: https://messerlab.org/slim/

### Key Papers:
- Browning & Browning (2011). "A fast, powerful method for detecting identity by descent."
- Palamara et al. (2012). "Length distributions of identity by descent reveal fine-scale demographic history."
- Ralph & Coop (2013). "The geography of recent genetic ancestry across Europe."
- Kelleher et al. (2016). "Efficient coalescent simulation and genealogical analysis for large sample sizes."